# Overview

It's common for a notebook to contain a cell that explicitly creates a new figure with automatic
number. When such as a cell is re-executed, `matplotlib` creates a new figure object. This is
convenience but is merely an illusion that *"hey, the cell output has changed, replacing the old
content"*. Unbeknowingly though, the old, unseen figure objects are still accumulated in the memory,
and they're *practically* orphan objects (bar some exceptional or advance users who need to access
those old figures).

This notebook simulates this issue, and demonstrate a possible best-effort hack to prevent orphan
figure objects when a cell is re-executed.

**Dependencies:**

- `%matplotlib widget` => `ipympl`
- `%autoreload 3` => `ipython>=8.*`
- colored prints => `rich` (optional)

In [ ]:
%matplotlib widget
%load_ext autoreload
%autoreload 3
%config IPythonBackend.figure_format = 'retina'

import contextlib

import matplotlib.pyplot as plt
import numpy as np

try:
    import rich

    rich.reconfigure(force_terminal=True, force_jupyter=False)
    rich.pretty.install()
    print = rich.get_console().out
except:
    pass


def figure2(*args, **kwargs):
    """Clear pre-existing figure when ``num = None``."""
    num = None
    if ("num" in kwargs) or (len(args) > 0):
        num = kwargs["num"] if "num" in kwargs else args[0]

    if num is not None:
        # Close pre-existing figures. Edge cases: there can be many figures with empty-string
        # labels, which were figures created with num=None.
        while plt.fignum_exists(num):
            fig = plt.figure(num=num)
            plt.close(fig)
            del fig

    return plt.figure(*args, **kwargs)


@contextlib.contextmanager
def figure_context(*args, **kwargs):
    """Clear pre-existing figure when ``num = None``."""
    yield figure2(*args, **kwargs)


def test_context(num=None):
    print("BEFORE:", plt.get_fignums(), plt.get_figlabels())
    with figure_context(num) as fig:
        plt.plot(np.sin(np.linspace(0, 10, 100)))
    print("AFTER:", plt.get_fignums(), plt.get_figlabels())


def test_figure2(num=None):
    print("BEFORE:", plt.get_fignums(), plt.get_figlabels())
    figure2(num)
    plt.plot(np.sin(np.linspace(0, 10, 100)))
    print("AFTER :", plt.get_fignums(), plt.get_figlabels())


def test(fun):
    with plt.ioff():
        # Reentrant (cell-level)
        print("Clear all existing figure, to begin test from clean slate.\n")
        for i in plt.get_fignums():
            plt.close(i)

        for i in (None, None, None, "figure_haha", "figure_haha", "plot_hehe", 4, 4, 5, 5, ""):
            s = f"'{i}'" if isinstance(i, str) else i
            print("(RE)CREATING FIGURE:", s)
            fun(i)
            print()

In [ ]:
test(test_figure2)

Clear all existing figure, to begin test from clean slate.

(RE)CREATING FIGURE: None
BEFORE: [] []
AFTER : [1] ['']

(RE)CREATING FIGURE: None
BEFORE: [1] ['']
AFTER : [1, 2] ['', '']

(RE)CREATING FIGURE: None
BEFORE: [1, 2] ['', '']
AFTER : [1, 2, 3] ['', '', '']

(RE)CREATING FIGURE: 'figure_haha'
BEFORE: [1, 2, 3] ['', '', '']
AFTER : [1, 2, 3, 4] ['', '', '', 'figure_haha']

(RE)CREATING FIGURE: 'figure_haha'
BEFORE: [1, 2, 3, 4] ['', '', '', 'figure_haha']
AFTER : [1, 2, 3, 4] ['', '', '', 'figure_haha']

(RE)CREATING FIGURE: 'plot_hehe'
BEFORE: [1, 2, 3, 4] ['', '', '', 'figure_haha']
AFTER : [1, 2, 3, 4, 5] ['', '', '', 'figure_haha', 'plot_hehe']

(RE)CREATING FIGURE: 4
BEFORE: [1, 2, 3, 4, 5] ['', '', '', 'figure_haha', 'plot_hehe']
AFTER : [1, 2, 3, 4, 5] ['', '', '', '', 'plot_hehe']

(RE)CREATING FIGURE: 4
BEFORE: [1, 2, 3, 4, 5] ['', '', '', '', 'plot_hehe']
AFTER : [1, 2, 3, 4, 5] ['', '', '', '', 'plot_hehe']

(RE)CREATING FIGURE: 5
BEFORE: [1, 2, 3, 4, 5] ['', '', '',

In [ ]:
test(test_context)

Clear all existing figure, to begin test from clean slate.

(RE)CREATING FIGURE: None
BEFORE: [] []
AFTER: [1] ['']

(RE)CREATING FIGURE: None
BEFORE: [1] ['']
AFTER: [1, 2] ['', '']

(RE)CREATING FIGURE: None
BEFORE: [1, 2] ['', '']
AFTER: [1, 2, 3] ['', '', '']

(RE)CREATING FIGURE: 'figure_haha'
BEFORE: [1, 2, 3] ['', '', '']
AFTER: [1, 2, 3, 4] ['', '', '', 'figure_haha']

(RE)CREATING FIGURE: 'figure_haha'
BEFORE: [1, 2, 3, 4] ['', '', '', 'figure_haha']
AFTER: [1, 2, 3, 4] ['', '', '', 'figure_haha']

(RE)CREATING FIGURE: 'plot_hehe'
BEFORE: [1, 2, 3, 4] ['', '', '', 'figure_haha']
AFTER: [1, 2, 3, 4, 5] ['', '', '', 'figure_haha', 'plot_hehe']

(RE)CREATING FIGURE: 4
BEFORE: [1, 2, 3, 4, 5] ['', '', '', 'figure_haha', 'plot_hehe']
AFTER: [1, 2, 3, 4, 5] ['', '', '', '', 'plot_hehe']

(RE)CREATING FIGURE: 4
BEFORE: [1, 2, 3, 4, 5] ['', '', '', '', 'plot_hehe']
AFTER: [1, 2, 3, 4, 5] ['', '', '', '', 'plot_hehe']

(RE)CREATING FIGURE: 5
BEFORE: [1, 2, 3, 4, 5] ['', '', '', '', 'pl